# ਸਧਾਰਨ ਚਿੱਤਰ ਵਰਗੀਕਰਨ

ਇਹ ਨੋਟਬੁੱਕ ਤੁਹਾਨੂੰ ਇੱਕ ਪਹਿਲਾਂ ਤੋਂ ਸਿਖਲਾਈ ਗਈ ਨਿਊਰਲ ਨੈਟਵਰਕ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਚਿੱਤਰਾਂ ਨੂੰ ਵਰਗਬੱਧ ਕਰਨ ਦਾ ਤਰੀਕਾ ਦਿਖਾਉਂਦੀ ਹੈ।

**ਤੁਹਾਨੂੰ ਕੀ ਸਿੱਖਣ ਨੂੰ ਮਿਲੇਗਾ:**
- ਪਹਿਲਾਂ ਤੋਂ ਸਿਖਲਾਈ ਗਏ ਮਾਡਲ ਨੂੰ ਕਿਵੇਂ ਲੋਡ ਅਤੇ ਵਰਤਣਾ ਹੈ
- ਚਿੱਤਰਾਂ ਦੀ ਪੂਰਵ-ਸੰਸਕਰਨ
- ਚਿੱਤਰਾਂ 'ਤੇ ਅਨੁਮਾਨ ਲਗਾਉਣਾ
- ਭਰੋਸੇਮੰਦ ਸਕੋਰਾਂ ਨੂੰ ਸਮਝਣਾ

**ਵਰਤੋਂ ਦਾ ਕੇਸ:** ਚਿੱਤਰਾਂ ਵਿੱਚ ਵਸਤੂਆਂ ਦੀ ਪਛਾਣ ਕਰਨਾ (ਜਿਵੇਂ "ਬਿੱਲੀ", "ਕੁੱਤਾ", "ਕਾਰ" ਆਦਿ)


## ਪਹਲਾ ਕਦਮ: ਲੋੜੀਂਦੇ ਲਾਇਬ੍ਰੇਰੀਆਂ ਇਮਪੋਰਟ ਕਰੋ

ਆਓ ਉਹ ਸਾਧਨ ਇਮਪੋਰਟ ਕਰੀਏ ਜਿਨ੍ਹਾਂ ਦੀ ਸਾਨੂੰ ਲੋੜ ਹੈ। ਚਿੰਤਾ ਨਾ ਕਰੋ ਜੇ ਤੁਸੀਂ ਅਜੇ ਇਹਨਾਂ ਨੂੰ ਪੂਰੀ ਤਰ੍ਹਾਂ ਨਹੀਂ ਸਮਝਦੇ!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ਚਰਨ 2: ਪਹਿਲਾਂ ਤੋਂ ਸਿਖਲਾਈ ਗਈ ਮਾਡਲ ਲੋਡ ਕਰੋ

ਅਸੀਂ **MobileNetV2** ਵਰਤਾਂਗੇ, ਜੋ ਪਹਿਲਾਂ ਹੀ ਲੱਖਾਂ ਤਸਵੀਰਾਂ 'ਤੇ ਸਿਖਲਾਈ ਗਈ ਹੈ।

ਇਸਨੂੰ **ਟ੍ਰਾਂਸਫਰ ਲਰਨਿੰਗ** ਕਿਹਾ ਜਾਂਦਾ ਹੈ - ਕਿਸੇ ਹੋਰ ਦੁਆਰਾ ਸਿਖਲਾਈ ਗਈ ਮਾਡਲ ਦੀ ਵਰਤੋਂ!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ਚਰਨ 3: ਸਹਾਇਕ ਫੰਕਸ਼ਨ

ਆਓ ਆਪਣੇ ਮਾਡਲ ਲਈ ਚਿੱਤਰਾਂ ਨੂੰ ਲੋਡ ਅਤੇ ਤਿਆਰ ਕਰਨ ਲਈ ਫੰਕਸ਼ਨ ਬਣਾਈਏ।


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ਚੌਥਾ ਕਦਮ: ਨਮੂਨਾ ਚਿੱਤਰਾਂ 'ਤੇ ਟੈਸਟ ਕਰੋ

ਆਓ ਇੰਟਰਨੈਟ ਤੋਂ ਕੁਝ ਚਿੱਤਰਾਂ ਦੀ ਵਰਗੀਕਰਨ ਕਰਨ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੀਏ!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### ਹਰ ਚਿੱਤਰ ਨੂੰ ਵਰਗਬੱਧ ਕਰੋ


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ਚਰਣ 5: ਆਪਣੀਆਂ ਤਸਵੀਰਾਂ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੋ!

ਹੇਠਾਂ ਦਿੱਤੇ URL ਨੂੰ ਕਿਸੇ ਵੀ ਤਸਵੀਰ ਦੇ URL ਨਾਲ ਬਦਲੋ ਜਿਸਨੂੰ ਤੁਸੀਂ ਵਰਗਬੱਧ ਕਰਨਾ ਚਾਹੁੰਦੇ ਹੋ।


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 ਕੀ ਹੋਇਆ?

1. **ਅਸੀਂ ਇੱਕ ਪੂਰਵ-ਤਿਆਰ ਮਾਡਲ ਲੋਡ ਕੀਤਾ** - MobileNetV2 ਨੂੰ ਲੱਖਾਂ ਤਸਵੀਰਾਂ 'ਤੇ ਟ੍ਰੇਨ ਕੀਤਾ ਗਿਆ ਸੀ
2. **ਅਸੀਂ ਤਸਵੀਰਾਂ ਨੂੰ ਪ੍ਰੀ-ਪ੍ਰੋਸੈਸ ਕੀਤਾ** - ਮਾਡਲ ਲਈ ਸਾਈਜ਼ ਅਤੇ ਫਾਰਮੈਟ ਕੀਤਾ
3. **ਮਾਡਲ ਨੇ ਅਨੁਮਾਨ ਲਗਾਏ** - ਇਸ ਨੇ 1000 ਵਸਤੂ ਵਰਗਾਂ ਲਈ ਸੰਭਾਵਨਾਵਾਂ ਨਿਕਾਲੀਆਂ
4. **ਅਸੀਂ ਨਤੀਜੇ ਡਿਕੋਡ ਕੀਤੇ** - ਗਿਣਤੀਆਂ ਨੂੰ ਮਨੁੱਖੀ-ਪੜ੍ਹਨਯੋਗ ਲੇਬਲਾਂ ਵਿੱਚ ਬਦਲਿਆ

### ਭਰੋਸੇਮੰਦ ਸਕੋਰਾਂ ਨੂੰ ਸਮਝਣਾ

- **90-100%**: ਬਹੁਤ ਭਰੋਸੇਮੰਦ (ਲਗਭਗ ਪੂਰੀ ਤਰ੍ਹਾਂ ਸਹੀ)
- **70-90%**: ਭਰੋਸੇਮੰਦ (ਸ਼ਾਇਦ ਸਹੀ)
- **50-70%**: ਕੁਝ ਹੱਦ ਤੱਕ ਭਰੋਸੇਮੰਦ (ਸਹੀ ਹੋ ਸਕਦਾ ਹੈ)
- **50% ਤੋਂ ਘੱਟ**: ਘੱਟ ਭਰੋਸੇਮੰਦ (ਅਸਪਸ਼ਟ)

### ਅਨੁਮਾਨ ਗਲਤ ਕਿਉਂ ਹੋ ਸਕਦੇ ਹਨ?

- **ਅਜੀਬ ਕੋਣ ਜਾਂ ਰੋਸ਼ਨੀ** - ਮਾਡਲ ਨੂੰ ਆਮ ਤਸਵੀਰਾਂ 'ਤੇ ਟ੍ਰੇਨ ਕੀਤਾ ਗਿਆ ਸੀ
- **ਕਈ ਵਸਤੂਆਂ** - ਮਾਡਲ ਇੱਕ ਮੁੱਖ ਵਸਤੂ ਦੀ ਉਮੀਦ ਕਰਦਾ ਹੈ
- **ਅਲਭੁਤ ਵਸਤੂਆਂ** - ਮਾਡਲ ਸਿਰਫ 1000 ਸ਼੍ਰੇਣੀਆਂ ਨੂੰ ਜਾਣਦਾ ਹੈ
- **ਘੱਟ ਗੁਣਵੱਤਾ ਵਾਲੀ ਤਸਵੀਰ** - ਧੁੰਦਲੀ ਜਾਂ ਪਿਕਸਲ ਵਾਲੀਆਂ ਤਸਵੀਰਾਂ ਨੂੰ ਸਮਝਣਾ ਮੁਸ਼ਕਲ ਹੁੰਦਾ ਹੈ

---


## 🚀 ਅਗਲੇ ਕਦਮ

1. **ਵੱਖ-ਵੱਖ ਚਿੱਤਰਾਂ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰੋ:**
   - [Unsplash](https://unsplash.com) 'ਤੇ ਚਿੱਤਰ ਲੱਭੋ
   - ਸੱਜੇ ਕਲਿੱਕ ਕਰੋ → "Copy image address" ਨਾਲ URL ਪ੍ਰਾਪਤ ਕਰੋ

2. **ਤਜਰਬਾ ਕਰੋ:**
   - ਅਬਸਟਰੈਕਟ ਆਰਟ ਨਾਲ ਕੀ ਹੁੰਦਾ ਹੈ?
   - ਕੀ ਇਹ ਵੱਖ-ਵੱਖ ਕੋਣਾਂ ਤੋਂ ਵਸਤੂਆਂ ਨੂੰ ਪਛਾਣ ਸਕਦਾ ਹੈ?
   - ਇਹ ਕਈ ਵਸਤੂਆਂ ਨੂੰ ਕਿਵੇਂ ਸੰਭਾਲਦਾ ਹੈ?

3. **ਹੋਰ ਸਿੱਖੋ:**
   - [Computer Vision ਪਾਠ](../lessons/4-ComputerVision/README.md) ਦੀ ਖੋਜ ਕਰੋ
   - ਆਪਣਾ ਚਿੱਤਰ ਵਰਗੀਕਰਕ ਤਿਆਰ ਕਰਨ ਦੀ ਸਿੱਖਿਆ ਲਵੋ
   - CNNs (Convolutional Neural Networks) ਕਿਵੇਂ ਕੰਮ ਕਰਦੇ ਹਨ, ਇਹ ਸਮਝੋ

---

## 🎉 ਵਧਾਈਆਂ!

ਤੁਸੀਂ ਇੱਕ ਅਧੁਨਿਕ ਨਿਊਰਲ ਨੈਟਵਰਕ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਚਿੱਤਰ ਵਰਗੀਕਰਕ ਤਿਆਰ ਕੀਤਾ ਹੈ!

ਇਹੀ ਤਕਨੀਕ ਇਹਨਾਂ ਨੂੰ ਸ਼ਕਤੀ ਦਿੰਦੀ ਹੈ:
- Google Photos (ਤੁਹਾਡੇ ਫੋਟੋਜ਼ ਨੂੰ ਸੰਗਠਿਤ ਕਰਨਾ)
- ਸਵੈ-ਚਲਿਤ ਕਾਰਾਂ (ਵਸਤੂਆਂ ਦੀ ਪਛਾਣ ਕਰਨਾ)
- ਮੈਡੀਕਲ ਨਿਰਣੇ (ਐਕਸ-ਰੇਜ਼ ਦਾ ਵਿਸ਼ਲੇਸ਼ਣ ਕਰਨਾ)
- ਗੁਣਵੱਤਾ ਨਿਯੰਤਰਣ (ਖਾਮੀਆਂ ਦੀ ਪਛਾਣ ਕਰਨਾ)

ਖੋਜ ਅਤੇ ਸਿੱਖਿਆ ਜਾਰੀ ਰੱਖੋ! 🚀



---

**ਅਸਵੀਕਰਤੀਕਰਨ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੱਜੇਪਣ ਹੋ ਸਕਦੇ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਅਸੀਂ ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
